Import files from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%set_env COLAB_GPU_TIMEOUT=24

env: COLAB_GPU_TIMEOUT=24


Args init

In [ ]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = Args(
    row_triples='/content/drive/MyDrive/Bachelor/FB15k/train.txt',
    amie_plus_path='/content/drive/MyDrive/Bachelor/amie-milestone-intKB.jar',
    minpca='0.5',
    minc='0.8',
    minhc='0.1',
#     new_prediction_path='./kaggle/working/new_prediction.tsv',
    new_prediction_path='/content/drive/MyDrive/Bachelor/Results/new_triple.txt',


    do_train=True,
    do_valid = True,
    do_test = True,
    evaluate_train = True,
    data_path='/content/drive/MyDrive/Bachelor/FB15k',
    model='RotatE',
    negative_sample_size=256,
    batch_size=1024,
    hidden_dim=1000,
    gamma=24.0,
    adversarial_temperature=1.0,
    negative_adversarial_sampling=True,
    learning_rate=0.0001,
    margin = None,
    max_steps=15000,
    max_steps_per = 2000,
    test_batch_size = 16,
    save_path='/content/drive/MyDrive/Bachelor/Results',
    double_entity_embedding=True,
    double_relation_embedding=False,
    countries=False,
    cpu_num=8,
    cuda=True,
    warm_up_steps=None,
    # init_checkpoint = '/content/drive/MyDrive/Bachelor/Results',
    init_checkpoint = None,
    uni_weight = False,
    regularization = 0.00003,
    # save_checkpoint_steps = 800,
    save_checkpoint_steps = 2000,
    log_steps = 100,
    valid_steps = 200,
    test_log_steps = 200,
    sparsity = False,
    conclusion_batch_size = 32,
    use_rule = True,
    NNE = True,
    use_rule_coefficient = 0.0001
)


Rule Mining

In [ ]:
import pandas
import subprocess
import os


def m_n_s(args, tsv_source_path, amie_path, minhc=0.01, minc=None, minpca=None):
    amie_arg = "java -XX:-UseGCOverheadLimit -Xmx4G -jar " + amie_path + " " + tsv_source_path

    if minhc != 0.01 and minhc:
        amie_arg += " -minhc "
        amie_arg += str(minhc)
    if minc:
        amie_arg += " -minc "
        amie_arg += str(minc)
    if minpca:
        amie_arg += " -minpca "
        amie_arg += str(minpca)

    output_directory = args.save_path
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    res_file = os.path.join(output_directory, 'result.txt')

    # tsv_target_path = "result_" + tsv_source_path
    open(res_file, 'w').close()     # clean up the old content in result.txt
    amie_arg += " >>"+res_file
    print(amie_arg)
    # to run the command in the shell
    os.system(amie_arg)
    print("AMIE Done")

    cols = [[] for x in range(11)]
    total_mined_rules = 0
    with open(res_file, 'r') as source:
        for line in source:
            if line[0] == '?':
                total_mined_rules += 1
                current_line = line.split()
                # print(current_line)
                if len(current_line) != 0:
                    current_rule = ''
                    if len(current_line) == 17:
                        for i in range(0, 10):
                            current_rule = current_rule + current_line[i] + ' '
                        cols[0].append(current_rule)

                        for j in range(10, 17):
                            if current_line[j][0] != '?':
                                cols[j - 9].append(float(current_line[j]))
                            else:
                                cols[j - 9].append(current_line[j])
                    else:
                        for i in range(0, 7):
                            current_rule = current_rule + current_line[i] + ' '
                        cols[0].append(current_rule)

                        for j in range(7, 14):
                            if current_line[j][0] != '?':
                                cols[j - 6].append(float(current_line[j]))
                            else:
                                cols[j - 6].append(current_line[j])

    df = pandas.DataFrame({
            'rule': cols[0],
            'v1': cols[1],
            'v2': cols[2],
            'v3': cols[3],
            'v4': cols[4],
            'v5': cols[5],
            'v6': cols[6],
            'v7': cols[7]
        })

    current_dir = os.path.abspath(os.path.dirname("__file__"))

    pca_sorted_path = os.path.join(current_dir, "pca_sorted_rule.tsv")
    std_sorted_path = os.path.join(current_dir, "std_sorted_rule.tsv")
    sorted_by_pca = df.sort_values(by='v3', ascending=False)
    sorted_by_std = df.sort_values(by='v2', ascending=False)
    sorted_by_pca.to_csv(pca_sorted_path, sep='\t', header=False, index= False)
    sorted_by_std.to_csv(std_sorted_path, sep='\t')

#     rules_df = df[['rule', 'v3']]

    # Saving the DataFrame to a file named "Rules.csv"
#     rules_df.to_csv('/kaggle/working/Rules.tsv', sep='\t', index=False, header=False)

#     return pca_sorted_path, std_sorted_path, total_mined_rules


Preprocess file

In [ ]:


def rule_transform(rule):
    rule_body, rule_head = rule.split('=>')
    rule_head = rule_head[5:-4]

    mark = ''

    if '?h' in rule_body:
        mark = '?h'

    if '?g' in rule_body:
        mark = '?g'

    if '?f' in rule_body:
        mark = '?f'

    if '?e' in rule_body:
        mark = '?e'

    if '?f' in rule_body:
        mark = '?f'


    if len(mark) > 0:
        r1 = ''
        r2 = ''
        rule_body_list = rule_body.split('  ')[:-1]

        ni_r1_qian = rule_body_list[0] == mark and rule_body_list[2] == '?a'
        shun_r1_qian = rule_body_list[0] == '?a' and rule_body_list[2] == mark
        ni_r1_hou =rule_body_list[3] == mark and rule_body_list[5] == '?a'
        shun_r1_hou = rule_body_list[3] == '?a' and rule_body_list[5] == mark

        ni_r2_qian = rule_body_list[0] == '?b' and rule_body_list[2] == mark
        shun_r2_qian = rule_body_list[0] == mark and rule_body_list[2] == '?b'
        ni_r2_hou = rule_body_list[3] == '?b' and rule_body_list[5] == mark
        shun_r2_hou = rule_body_list[3] == mark and rule_body_list[5] == '?b'

        if ni_r2_qian and shun_r1_hou:
            r2 = '-' + rule_body_list[1]
            r1 = rule_body_list[4]
        elif shun_r2_qian and shun_r1_hou:
            r2 = rule_body_list[1]
            r1 = rule_body_list[4]
        elif ni_r2_hou and shun_r1_qian:
            r2 = '-' + rule_body_list[4]
            r1 = rule_body_list[1]
        elif shun_r2_hou and shun_r1_qian:
            r2 = rule_body_list[4]
            r1 = rule_body_list[1]
        elif ni_r1_hou and ni_r2_qian:
            r1 = '-' + rule_body_list[4]
            r2 = '-' + rule_body_list[1]
        elif ni_r1_hou and shun_r2_qian:
            r1 = '-' + rule_body_list[4]
            r2 = rule_body_list[1]
        elif ni_r1_qian and ni_r2_hou:
            r2 = '-' + rule_body_list[4]
            r1 = '-' + rule_body_list[1]
        elif ni_r1_qian and shun_r2_hou:
            r1 = '-' + rule_body_list[1]
            r2 = rule_body_list[4]

        return r1 + ',' + r2 + ',' + rule_head

    else:
        rule_body_list = rule_body.split('  ')[:-1]
        if len(rule_body.split('  ')) >= 7:
            ni__qian = rule_body_list[0] == '?b' and rule_body_list[2] == '?a'
            shun__qian = rule_body_list[0] == '?a' and rule_body_list[2] == '?b'
            ni__hou = rule_body_list[3] == '?b' and rule_body_list[5] == '?a'
            shun__hou = rule_body_list[3] == '?a' and rule_body_list[5] == '?b'
            if ni__qian and shun__hou:
                r2 = rule_body_list[1]
                r1 = rule_body_list[4]
            elif shun__qian and shun__hou:
                r1 = rule_body_list[1]
                r2 = '-' + rule_body_list[4]
            elif ni__hou and shun__qian:
                r2 = rule_body_list[4]
                r1 = rule_body_list[1]
            elif ni__hou and ni__qian:
                r2 = rule_body_list[4]
                r1 = '-' + rule_body_list[1]
            return r1 + ',' + r2 + ',' + rule_head
        else:
            if rule_body[:2] == '?a':
                return rule_body[4:-7] + ',' + rule_head
            elif rule_body[:2] == '?b':
                return '-' + rule_body[4:-7] + ',' + rule_head




def preprocess():

    rule_file = os.path.join(args.save_path, 'result.txt')
    rule_file2 = os.path.join(args.save_path, 'Rules.txt')

    with open(rule_file, 'r') as f:
        with open(rule_file2, 'w') as f2:
            for line in f:
                # if it begins with ?
                if line[0] != '?':
                    continue
                Rule, Head_Coverage, Std_Confidence, PCA_Confidence, Positive_Examples, Body_size, PCA_Body_size, _ \
                    = line.strip().split('\t')

                # Rule, PCA_Confidence  = line.strip().split('\t')

                rule_str = rule_transform(Rule)
                newline = rule_str + '\t' + str(PCA_Confidence) + '\n'

                f2.write(newline)

Rule Reasoning

In [ ]:
from collections import defaultdict


def one_time_reasoning_for_dense(entity, rules, new_tri, in_map, out_map):
    for rule_index, (rule, p) in enumerate(rules):
        if len(rule) == 3:
            if rule[0].startswith('-'):
                e2_list = in_map[(rule[0][1:], entity)]
            else:
                e2_list = out_map[(entity, rule[0])]

            if len(e2_list) > 0:
                for e2 in e2_list:
                    if rule[1].startswith('-'):
                        e3_list = in_map[(rule[1][1:], e2)]
                    else:
                        e3_list = out_map[(e2, rule[1])]
                    for e3 in e3_list:
                        new_tri.add((entity, rule[2], e3))
            else:
                continue


        elif len(rule) == 2:

            if rule[0].startswith('-'):
                e2_list = in_map[(rule[0][1:], entity)]
            else:
                e2_list = out_map[(entity, rule[0])]

            for e2 in e2_list:
                new_tri.add((entity, rule[1], e2))


def one_time_reasoning(entity, rules, new_tri, in_map, out_map):
    for e1 in entity:
        for rule_index, (rule, p) in enumerate(rules):
            if len(rule) == 3:
                if rule[0].startswith('-'):
                    e2_list = in_map[(rule[0][1:], e1)]
                else:
                    e2_list = out_map[(e1, rule[0])]

                if len(e2_list) > 0:
                    for e2 in e2_list:
                        if rule[1].startswith('-'):
                            e3_list = in_map[(rule[1][1:], e2)]
                        else:
                            e3_list = out_map[(e2, rule[1])]
                        for e3 in e3_list:
                            new_tri.add((e1, rule[2], e3, p))
                else:
                    continue


            elif len(rule) == 2:

                if rule[0].startswith('-'):
                    e2_list = in_map[(rule[0][1:], e1)]
                else:
                    e2_list = out_map[(e1, rule[0])]

                for e2 in e2_list:
                    new_tri.add((e1, rule[1], e2, p))




def dense_your_graph(input, rules):
    new_triples = set()
    out_map = defaultdict(list)
    in_map = defaultdict(list)
    entity = set()

    with open(input, 'r', encoding='utf8') as f:
        for line in f:
            e1, r, e2 = line.strip().split("\t")
            out_map[(e1, r)].append(e2)
            in_map[(r, e2)].append(e1)
            entity.add(e1)
            entity.add(e2)

    entity = list(entity)

    for i in entity:
        one_time_reasoning_for_dense(i, rules, new_triples, in_map, out_map)

    return new_triples


# if __name__ == '__main__':
def rule_reasoning(args):

    input_tri = args.row_triples

    train_set = set()

    with open(input_tri) as f_train:
        for line in f_train:
            h, r, t = line.strip().split('\t')
            train_set.add((h, r, t))


    out_new_triples = args.new_prediction_path
    rules_path = os.path.join(args.save_path, 'Rules.txt')

    rules = []

    with open(rules_path, 'r', encoding='utf8') as f:
        for line in f:
            rule, p = line.strip().split('\t')
            rules.append(tuple([tuple(rule.split(',')), float(p)]))

    f_out = open(out_new_triples, 'w')

    for i in range(len(rules)):

        newline = ''
        rules2 = rules[i:i + 1]
        _, p = rules2[0]
        newline += str(p) + '\t'
        new_triples = dense_your_graph(input_tri, rules2)
        new_triples = new_triples - train_set


        for h, r, t in new_triples:
            newline += h + '@$' + r + '@$' + t + '\t'
        newline = newline[:-1] + '\n'

        if len(new_triples) > 0:
            f_out.write(newline)

Run file

In [ ]:
#!/usr/bin/python3

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function



import argparse
import json
import logging
import os
import random

import numpy as np
import torch

from torch.utils.data import DataLoader



from collections import defaultdict
def parse_args(args=None):
    parser = argparse.ArgumentParser(
        description='Training and Testing Knowledge Graph Embedding Models',
        usage='train.py [<args>] [-h | --help]'
    )

    parser.add_argument('--cuda', action='store_true', help='use GPU')

    parser.add_argument('-ur', '--use_rule', action='store_true')
    parser.add_argument('-urc', '--use_rule_coefficient', default=0.0001, type=float)
    parser.add_argument('--sparsity', action='store_true')
    parser.add_argument('--NNE', action='store_true')
    parser.add_argument('-cbs', '--conclusion_batch_size', default=32, type=int)
    parser.add_argument("--max_steps_per", default=40000, type=int)

    parser.add_argument('--do_train', action='store_true')
    parser.add_argument('--do_valid', action='store_true')
    parser.add_argument('--do_test', action='store_true')
    parser.add_argument('--evaluate_train', action='store_true', help='Evaluate on training data')

    parser.add_argument('--countries', action='store_true', help='Use Countries S1/S2/S3 datasets')
    parser.add_argument('--regions', type=int, nargs='+', default=None,
                        help='Region Id for Countries S1/S2/S3 datasets, DO NOT MANUALLY SET')

    parser.add_argument('--data_path', type=str, default=None)
    parser.add_argument('--model', default='TransE', type=str)
    parser.add_argument('-de', '--double_entity_embedding', action='store_true')
    parser.add_argument('-dr', '--double_relation_embedding', action='store_true')



    parser.add_argument('-n', '--negative_sample_size', default=128, type=int)
    parser.add_argument('-d', '--hidden_dim', default=500, type=int)
    parser.add_argument('-g', '--gamma', default=12.0, type=float)
    parser.add_argument('-adv', '--negative_adversarial_sampling', action='store_true')
    parser.add_argument('-a', '--adversarial_temperature', default=1.0, type=float)
    parser.add_argument('-b', '--batch_size', default=1024, type=int)
    parser.add_argument('-r', '--regularization', default=0.0, type=float)
    parser.add_argument('--test_batch_size', default=4, type=int, help='valid/test batch size')
    parser.add_argument('--uni_weight', action='store_true',
                        help='Otherwise use subsampling weighting like in word2vec')

    parser.add_argument('-lr', '--learning_rate', default=0.0001, type=float)
    parser.add_argument('-cpu', '--cpu_num', default=10, type=int)
    parser.add_argument('-init', '--init_checkpoint', default=None, type=str)
    parser.add_argument('-save', '--save_path', default=None, type=str)
    parser.add_argument('--max_steps', default=300000, type=int)
    parser.add_argument('--warm_up_steps', default=None, type=int)

    parser.add_argument('--save_checkpoint_steps', default=10000, type=int)
    parser.add_argument('--valid_steps', default=10000, type=int)
    parser.add_argument('--log_steps', default=100, type=int, help='train log every xx steps')
    parser.add_argument('--test_log_steps', default=1000, type=int, help='valid/test log every xx steps')

    parser.add_argument('--nentity', type=int, default=0, help='DO NOT MANUALLY SET')
    parser.add_argument('--nrelation', type=int, default=0, help='DO NOT MANUALLY SET')

    return parser.parse_args(args)

def override_config(args):
    '''
    Override model and data configuration
    '''

    with open(os.path.join(args.init_checkpoint, 'config.json'), 'r') as fjson:
        argparse_dict = json.load(fjson)

    args.countries = argparse_dict['countries']
    if args.data_path is None:
        args.data_path = argparse_dict['data_path']
    args.model = argparse_dict['model']
    args.double_entity_embedding = argparse_dict['double_entity_embedding']
    args.double_relation_embedding = argparse_dict['double_relation_embedding']
    args.hidden_dim = argparse_dict['hidden_dim']
    args.test_batch_size = argparse_dict['test_batch_size']


def save_model(model, optimizer, save_variable_list, args):
    '''
    Save the parameters of the model and the optimizer,
    as well as some other variables such as step and learning_rate
    '''

    argparse_dict = vars(args)
    with open(os.path.join(args.save_path, 'config.json'), 'w') as fjson:
        json.dump(argparse_dict, fjson)

    print("My variable list: ", save_variable_list)

    torch.save({
        **save_variable_list,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()},
        os.path.join(args.save_path, 'checkpoint')
    )

    entity_embedding = model.entity_embedding.detach().cpu().numpy()
    np.save(
        os.path.join(args.save_path, 'entity_embedding'),
        entity_embedding
    )

    relation_embedding = model.relation_embedding.detach().cpu().numpy()
    np.save(
        os.path.join(args.save_path, 'relation_embedding'),
        relation_embedding
    )

def read_triple(file_path, entity2id, relation2id):
    '''
    Read triples and map them into ids.
    '''
    triples = []
    with open(file_path) as fin:
        for line in fin:
            h, r, t = line.strip().split('\t')
            triples.append((entity2id[h], relation2id[r], entity2id[t]))
    return triples

def read_triple2(file_path, entity2id, relation2id):
    '''
    Read triples and map them into ids.
    '''
    triples = []
    with open(file_path) as fin:
        for line in fin:
            h, t, r = line.strip().split('\t')
            triples.append((entity2id[h], relation2id[r], entity2id[t]))
    return triples

def set_logger(args):
    '''
    Write logs to checkpoint and console
    '''

    if args.do_train:
        log_file = os.path.join(args.save_path or args.init_checkpoint, 'train.log')
    else:
        log_file = os.path.join(args.save_path or args.init_checkpoint, 'test.log')
# hello
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        filename=log_file,
        filemode='w'
    )
    console = logging.StreamHandler()
    console.setLevel(logging.CRITICAL)
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

def log_metrics(mode, step, metrics):
    '''
    Print the evaluation logs
    '''
    for metric in metrics:
        if metric == 'rule triple score:':
            logging.critical('%s %s at step %d: \n %s' % (mode, metric, step, str(metrics[metric])))
        else:
            logging.critical('%s %s at step %d: %f' % (mode, metric, step, metrics[metric]))


def runKGE(args):
    if (not args.do_train) and (not args.do_valid) and (not args.do_test):
        raise ValueError('one of train/val/test mode must be choosed.')

    if args.init_checkpoint:
        override_config(args)
    elif args.data_path is None:
        raise ValueError('one of init_checkpoint/data_path must be choosed.')

    if args.do_train and args.save_path is None:
        raise ValueError('Where do you want to save your trained model?')

    if args.save_path and not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

    # Write logs to checkpoint and console
    set_logger(args)




    rules_path = os.path.join(args.save_path, 'Rules.txt')
#     rules_path = '/kaggle/input/myoutput/Rules.txt'
    rules = []
    with open(rules_path, 'r', encoding='utf8') as f:
        for line in f:
            rule, p = line.strip().split('\t')
            rules.append(tuple([tuple(rule.split(',')), float(p)]))


    with open(os.path.join(args.data_path, 'entities.dict')) as fin:
        entity2id = dict()
        id2entity = dict()
        for line in fin:
            eid, entity = line.strip().split('\t')
            entity2id[entity] = int(eid)
            id2entity[int(eid)] = entity

    with open(os.path.join(args.data_path, 'relations.dict')) as fin:
        relation2id = dict()
        id2relation = dict()
        for line in fin:
            rid, relation = line.strip().split('\t')
            relation2id[relation] = int(rid)
            id2relation[int(rid)] = relation

    # Read regions for Countries S* datasets
    if args.countries:
        regions = list()
        with open(os.path.join(args.data_path, 'regions.list')) as fin:
            for line in fin:
                region = line.strip()
                regions.append(entity2id[region])
        args.regions = regions

    nentity = len(entity2id)
    nrelation = len(relation2id)

    args.nentity = nentity
    args.nrelation = nrelation
    print('Model', args.model)
    logging.critical('Model: %s' % args.model)
    logging.critical('Data Path: %s' % args.data_path)
    logging.critical('#entity: %d' % nentity)
    logging.critical('#relation: %d' % nrelation)

    train_triples = read_triple(os.path.join(args.data_path, 'train.txt'), entity2id, relation2id)
    logging.critical('#train: %d' % len(train_triples))
    valid_triples = read_triple(os.path.join(args.data_path, 'valid.txt'), entity2id, relation2id)

    logging.critical('#valid: %d' % len(valid_triples))
    test_triples = read_triple(os.path.join(args.data_path, 'test.txt'), entity2id, relation2id)
    logging.critical('#test: %d' % len(test_triples))

    #All true triples
    all_true_triples = train_triples + valid_triples + test_triples
    if args.sparsity:
        sparse_triples = read_triple2(os.path.join(args.data_path, 'test_sparsity_0.995.txt'), entity2id, relation2id)
        all_true_triples = train_triples + valid_triples + sparse_triples






    def conclusion_filtering(entity_embedding, relation_embedding, embedding_range, gamma, new_triple):

        def RotatE_score(head, relation, tail):
            pi = 3.14159265358979323846

            re_head, im_head = torch.chunk(head, 2, dim=0)
            re_tail, im_tail = torch.chunk(tail, 2, dim=0)

            # Make phases of relations uniformly distributed in [-pi, pi]

            phase_relation = relation / (embedding_range.item() / pi)

            re_relation = torch.cos(phase_relation)
            im_relation = torch.sin(phase_relation)

            re_score = re_head * re_relation - im_head * im_relation
            im_score = re_head * im_relation + im_head * re_relation
            re_score = re_score - re_tail
            im_score = im_score - im_tail

            score = torch.stack([re_score, im_score], dim=0)
            score = score.norm(dim=0)

            score = gamma.item() - score.sum(dim=0)

            # re_head, im_head = torch.chunk(head, 2, dim=0)
            # re_relation, im_relation = torch.chunk(relation, 2, dim=0)

            # re_tail, im_tail = torch.chunk(tail, 2, dim=0)
            # re_score = re_head * re_relation - im_head * im_relation
            # im_score = re_head * im_relation + im_head * re_relation
            # score = re_score * re_tail + im_score * im_tail
            # score = score.sum(dim=0)
            return score
        true_conclusions = []
        for h,r,t in new_triple:
            head, relation, tail = entity_embedding[h], relation_embedding[r], entity_embedding[t]
            if torch.sigmoid(RotatE_score(head, relation, tail)) > 0.99:
                true_conclusions.append((h,r,t))
        return true_conclusions

    def more_triple_p(new_triple_path):
        rule_num = 0
        rule_triples_index = []
        new_triple_only = []
        with open(new_triple_path) as f:
            for line in f:
                rule_num += 1
                rule_triples = line.strip().split('\t')
                p = float(rule_triples[0])
                rule_triples = rule_triples[1:]

                for i in rule_triples:
                    # h, r, t = i.split(' ')
                    h, r, t = i.split('@$')

                    rule_triples_index.append((entity2id[h], relation2id[r], entity2id[t], p))
                    new_triple_only.append((entity2id[h], relation2id[r], entity2id[t]))

        return rule_triples_index, new_triple_only,rule_num


    new_triple, new_triple_only,rule_num = more_triple_p(args.new_prediction_path)
#     new_triple, new_triple_only,rule_num = more_triple_p('/kaggle/input/myoutput/new_triple.txt')



    kge_model = KGEModel(
        model_name=args.model,
        nentity=nentity,
        nrelation=nrelation,
        hidden_dim=args.hidden_dim,
        gamma=args.gamma,
        double_entity_embedding=args.double_entity_embedding,
        double_relation_embedding=args.double_relation_embedding
    )

    logging.critical('Model Parameter Configuration:')
    for name, param in kge_model.named_parameters():
        logging.critical('Parameter %s: %s, require_grad = %s' % (name, str(param.size()), str(param.requires_grad)))

    if args.cuda:
        kge_model = kge_model.cuda()

    if args.do_train:
        # Set training dataloader iterator
        train_dataloader_head = DataLoader(
            TrainDataset(train_triples,new_triple_only, nentity, nrelation, args.negative_sample_size, 'head-batch'),
            batch_size=args.batch_size,
            shuffle=True,
            # num_workers=max(1, args.cpu_num//2),
            num_workers=1,
            collate_fn=TrainDataset.collate_fn
        )

        train_dataloader_tail = DataLoader(
            TrainDataset(train_triples, new_triple_only,nentity,  nrelation, args.negative_sample_size, 'tail-batch'),
            batch_size=args.batch_size,
            shuffle=True,
            # num_workers=max(1, args.cpu_num//2),
            num_workers=1,
            collate_fn=TrainDataset.collate_fn
        )

        train_iterator = BidirectionalOneShotIterator(train_dataloader_head, train_dataloader_tail)


        train_rule_triple = DataLoader(
            RuleTriple(new_triple, nentity, nrelation),
            batch_size=args.conclusion_batch_size,
            shuffle=True,
            # num_workers=max(1, args.cpu_num//2),
            num_workers=1,
            collate_fn=RuleTriple.collate_fn
        )

        train_rule_triple_iterator = Iterator(train_rule_triple)



        # Set training configuration
        current_learning_rate = args.learning_rate
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, kge_model.parameters()),
            lr=current_learning_rate
        )
        if args.warm_up_steps:
            warm_up_steps = args.warm_up_steps
        else:
            warm_up_steps = args.max_steps // 2

    if args.init_checkpoint:
        # Restore model from checkpoint directory
        logging.critical('Loading checkpoint %s...' % args.init_checkpoint)
        checkpoint = torch.load(os.path.join(args.init_checkpoint, 'checkpoint'))
        init_step = checkpoint['step']
        print(os.path.join(args.init_checkpoint, 'checkpoint')) #hello
        kge_model.load_state_dict(checkpoint['model_state_dict'])
        if args.do_train:
            current_learning_rate = checkpoint['current_learning_rate']
            warm_up_steps = checkpoint['warm_up_steps']
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        logging.critical('Ramdomly Initializing %s Model...' % args.model)
        init_step = 0

    step = init_step






    logging.critical('Start Training...')
    logging.critical('init_step = %d' % init_step)
    logging.critical('batch_size = %d' % args.batch_size)
    logging.critical('negative_adversarial_sampling = %d' % args.negative_adversarial_sampling)
    logging.critical('hidden_dim = %d' % args.hidden_dim)
    logging.critical('gamma = %f' % args.gamma)
    logging.critical('negative_adversarial_sampling = %s' % str(args.negative_adversarial_sampling))

    if args.use_rule:
        logging.critical('rule_num = %d' % rule_num)
        logging.critical('new_triple_num = %d' % len(new_triple_only))

    if args.negative_adversarial_sampling:
        logging.critical('adversarial_temperature = %f' % args.adversarial_temperature)

    # Set valid dataloader as it would be evaluated during training
    best_hits1 = 0
    patience = 5  # Number of epochs to wait for improvement before stopping
    no_improvement_counter = 0

    if args.do_train:
        logging.critical('learning_rate = %f' % current_learning_rate)

        training_logs = []

        #Training Loop
        for step in range(init_step, args.max_steps):

            log = kge_model.train_step(kge_model, optimizer, train_iterator, train_rule_triple_iterator, args)

            training_logs.append(log)

            if step >= warm_up_steps:
                current_learning_rate = current_learning_rate / 10
                logging.critical('Change learning_rate to %f at step %d' % (current_learning_rate, step))
                optimizer = torch.optim.Adam(
                    filter(lambda p: p.requires_grad, kge_model.parameters()),
                    lr=current_learning_rate
                )
                warm_up_steps = warm_up_steps * 3

            if step % args.save_checkpoint_steps == 0:
                save_variable_list = {
                    'step': step,
                    'current_learning_rate': current_learning_rate,
                    'warm_up_steps': warm_up_steps
                }
                save_model(kge_model, optimizer, save_variable_list, args)

            if step % args.log_steps == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    if metric != "rule triple score:":
                        metrics[metric] = sum([log[metric] for log in training_logs])/len(training_logs)
                log_metrics('Training average', step, metrics)
                # training_logs = []

            if step % 1000 == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    if metric == "rule triple score:":
                        metrics[metric] = training_logs[-1][metric]
                log_metrics('Training average', step, metrics)
                training_logs = []

            if args.do_valid and step % args.valid_steps == 0 and step != 0:
                logging.critical('Evaluating on Valid Dataset...')
                metrics,_,__ = kge_model.test_step(kge_model, valid_triples, all_true_triples, args, id2entity, id2relation)
                log_metrics('Valid', step, metrics)
                print(metrics)
                hits1 = metrics['HITS@1']
                if hits1 > best_hits1:
                  best_hits1 = hits1
                  no_improvement_counter = 0
                else:
                  no_improvement_counter += 1

                # Check early stopping condition
                if no_improvement_counter >= patience:
                    logging.critical(f'Hits@1 did not improve for {patience} epochs. Early stopping...')
                    break


            if step % args.max_steps_per == 0:
                true_conclusions = conclusion_filtering(kge_model.entity_embedding.detach().cpu(),
                                               kge_model.relation_embedding.detach().cpu(),kge_model.embedding_range, kge_model.gamma,
                                               new_triple_only)
                train_triples = train_triples + true_conclusions
                train_dataloader_head = DataLoader(
                    TrainDataset(train_triples, new_triple_only, nentity, nrelation, args.negative_sample_size,
                                 'head-batch'),
                    batch_size=args.batch_size,
                    shuffle=True,
                    num_workers=max(1, args.cpu_num // 2),
                    collate_fn=TrainDataset.collate_fn
                )
                train_dataloader_tail = DataLoader(
                    TrainDataset(train_triples, new_triple_only, nentity, nrelation, args.negative_sample_size,
                                 'tail-batch'),
                    batch_size=args.batch_size,
                    shuffle=True,
                    num_workers=max(1, args.cpu_num // 2),
                    collate_fn=TrainDataset.collate_fn
                )
                train_iterator = BidirectionalOneShotIterator(train_dataloader_head, train_dataloader_tail)
                out_map = defaultdict(list)
                in_map = defaultdict(list)
                entity = set()
                for e1, r, e2 in train_triples:
                    actual_e1 = id2entity[e1]
                    actual_e2 = id2entity[e2]
                    actual_r = id2relation[r]
                    out_map[(actual_e1, actual_r)].append(actual_e2)
                    in_map[(actual_r, actual_e2)].append(actual_e1)
                    entity.add(actual_e1)
                    entity.add(actual_e2)
                    # out_map[(e1, r)].append(e2)
                    # in_map[(r, e2)].append(e1)
                    # entity.add(e1)
                    # entity.add(e2)
                entity = list(entity)
                conclusions = set()
                one_time_reasoning(entity, rules, conclusions, in_map, out_map)
                conclusions_list = list(conclusions)
                for i in range(len(conclusions_list)):
                    h, r, t, p = conclusions_list[i]
                    conclusions_list[i] = (entity2id[h], relation2id[r], entity2id[t], p)

#                 conclusions_list_filtered = []

#                 for triple in conclusions_list:
#                     triple_indices = triple[:3]
#                     if triple_indices not in train_triples:
#                         conclusions_list_filtered.append(triple)


                train_rule_triple = DataLoader(
                    RuleTriple(conclusions_list, nentity, nrelation),
                    batch_size=args.conclusion_batch_size,
                    shuffle=True,
                    num_workers=max(1, args.cpu_num // 2),
                    collate_fn=RuleTriple.collate_fn
                )
                train_rule_triple_iterator = Iterator(train_rule_triple)

        save_variable_list = {
            'step': step,
            'current_learning_rate': current_learning_rate,
            'warm_up_steps': warm_up_steps
        }
        save_model(kge_model, optimizer, save_variable_list, args)



    if args.do_valid:
        logging.critical('Evaluating on Valid Dataset...')
        metrics, head_triple, tail_triple = kge_model.test_step(kge_model, valid_triples, all_true_triples, args, id2entity, id2relation)
        log_metrics('Valid', step, metrics)

    if args.sparsity:
        logging.critical('Evaluating on sparsity Dataset...')
        metrics, head_triple, tail_triple = kge_model.test_step(kge_model, sparse_triples, all_true_triples, args,id2entity, id2relation)
        log_metrics('sparsity', step, metrics)

    if args.do_test:
        logging.critical('Evaluating on Test Dataset...')
        metrics, head_triple, tail_triple = kge_model.test_step(kge_model, test_triples, all_true_triples, args, id2entity, id2relation)
        log_metrics('Test', step, metrics)

    if args.evaluate_train:
        logging.critical('Evaluating on Training Dataset...')
        metrics,head_triple, tail_triple = kge_model.test_step(kge_model, train_triples, all_true_triples, args, id2entity, id2relation)
        log_metrics('Test', step, metrics)



# if __name__ == '__main__':
#     main(parse_args())

Model

In [ ]:
#!/usr/bin/python3

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import average_precision_score

from torch.utils.data import DataLoader


class KGEModel(nn.Module):
    def __init__(self, model_name, nentity, nrelation, hidden_dim, gamma,
                 double_entity_embedding=False, double_relation_embedding=False, e_emb=None, et_emb=None, r_emb=None):
        super(KGEModel, self).__init__()
        self.model_name = model_name
        self.nentity = nentity
        self.nrelation = nrelation
        self.hidden_dim = hidden_dim
        self.epsilon = 2.0

        self.gamma = nn.Parameter(
            torch.Tensor([gamma]),
            requires_grad=False
        )

        self.embedding_range = nn.Parameter(
            torch.Tensor([(self.gamma.item() + self.epsilon) / hidden_dim]),
            requires_grad=False
        )

        self.entity_dim = hidden_dim * 2 if double_entity_embedding else hidden_dim
        self.relation_dim = hidden_dim * 2 if double_relation_embedding else hidden_dim


        self.entity_embedding = nn.Parameter(torch.zeros(nentity, self.entity_dim))
        nn.init.uniform_(
            tensor=self.entity_embedding,
            a=-self.embedding_range.item(),
            b=self.embedding_range.item()
        )

        self.relation_embedding = nn.Parameter(torch.zeros(nrelation, self.relation_dim))
        nn.init.uniform_(
            tensor=self.relation_embedding,
            a=-self.embedding_range.item(),
            b=self.embedding_range.item()
        )

        # self.entity_embedding = nn.Parameter(e_emb, requires_grad=True)
        # self.relation_embedding = nn.Parameter(r_emb, requires_grad=True)

        # self.entity_dim = 400
        # self.relation_dim = 200
        # et_emb.requires_grad = True

        # self.entity_embedding = nn.Parameter(torch.zeros(nentity, 400))
        # nn.init.uniform_(
        #     tensor=self.entity_embedding,
        #     a=-self.embedding_range.item(),
        #     b=self.embedding_range.item()
        # )

        # self.entity_embedding = nn.Parameter((self.entity_embedding + et_emb)/2)
        #
        # self.entity_embedding = nn.Parameter(e_emb, requires_grad=False)
        # self.w = nn.Linear(800, 400)
        # self.entity_embedding = nn.Parameter(self.w(torch.cat([self.entity_embedding, et_emb], dim=1)))
        # self.relation_embedding = nn.Parameter(r_emb, requires_grad=True)

        if model_name == 'pRotatE':
            self.modulus = nn.Parameter(torch.Tensor([[0.5 * self.embedding_range.item()]]))

        # Do not forget to modify this line when you add a new model in the "forward" function
        if model_name not in ['TransE', 'DistMult', 'ComplEx', 'RotatE', 'pRotatE']:
            raise ValueError('model %s not supported' % model_name)

        if model_name == 'RotatE' and (not double_entity_embedding or double_relation_embedding):
            raise ValueError('RotatE should use --double_entity_embedding')

        if model_name == 'ComplEx' and (not double_entity_embedding or not double_relation_embedding):
            raise ValueError('ComplEx should use --double_entity_embedding and --double_relation_embedding')

    def forward(self, sample, mode='single'):
        '''
        Forward function that calculate the score of a batch of triples.
        In the 'single' mode, sample is a batch of triple.
        In the 'head-batch' or 'tail-batch' mode, sample consists two part.
        The first part is usually the positive sample.
        And the second part is the entities in the negative samples.
        Because negative samples and positive samples usually share two elements
        in their triple ((head, relation) or (relation, tail)).
        '''

        if mode == 'single':
            batch_size, negative_sample_size = sample.size(0), 1

            head = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=sample[:, 0]
            ).unsqueeze(1)

            relation = torch.index_select(
                self.relation_embedding,
                dim=0,
                index=sample[:, 1]
            ).unsqueeze(1)

            tail = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=sample[:, 2]
            ).unsqueeze(1)

        elif mode == 'head-batch':
            tail_part, head_part = sample
            batch_size, negative_sample_size = head_part.size(0), head_part.size(1)

            head = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=head_part.view(-1)
            ).view(batch_size, negative_sample_size, -1)

            relation = torch.index_select(
                self.relation_embedding,
                dim=0,
                index=tail_part[:, 1]
            ).unsqueeze(1)

            tail = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=tail_part[:, 2]
            ).unsqueeze(1)




        elif mode == 'tail-batch':
            head_part, tail_part = sample
            batch_size, negative_sample_size = tail_part.size(0), tail_part.size(1)

            head = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=head_part[:, 0]
            ).unsqueeze(1)

            relation = torch.index_select(
                self.relation_embedding,
                dim=0,
                index=head_part[:, 1]
            ).unsqueeze(1)

            tail = torch.index_select(
                self.entity_embedding,
                dim=0,
                index=tail_part.view(-1)
            ).view(batch_size, negative_sample_size, -1)

        else:
            raise ValueError('mode %s not supported' % mode)

        model_func = {
            'TransE': self.TransE,
            'DistMult': self.DistMult,
            'ComplEx': self.ComplEx,
            'RotatE': self.RotatE,
            'pRotatE': self.pRotatE
        }




        if self.model_name in model_func:
            score = model_func[self.model_name](head, relation, tail, mode)
        else:
            raise ValueError('model %s not supported' % self.model_name)

        return score

    def TransE(self, head, relation, tail, mode):
        if mode == 'head-batch':
            score = head + (relation - tail)
        else:
            score = (head + relation) - tail

        score = self.gamma.item() - torch.norm(score, p=1, dim=2)
        return score

    def DistMult(self, head, relation, tail, mode):
        if mode == 'head-batch':
            score = head * (relation * tail)
        else:
            score = (head * relation) * tail

        score = score.sum(dim=2)
        return score

    def ComplEx(self, head, relation, tail, mode):
        re_head, im_head = torch.chunk(head, 2, dim=2)
        re_relation, im_relation = torch.chunk(relation, 2, dim=2)
        re_tail, im_tail = torch.chunk(tail, 2, dim=2)

        if mode == 'head-batch':
            re_score = re_relation * re_tail + im_relation * im_tail
            im_score = re_relation * im_tail - im_relation * re_tail
            score = re_head * re_score + im_head * im_score
        else:
            re_score = re_head * re_relation - im_head * im_relation
            im_score = re_head * im_relation + im_head * re_relation
            score = re_score * re_tail + im_score * im_tail

        score = score.sum(dim=2)
        return score

    def RotatE(self, head, relation, tail, mode):
        pi = 3.14159265358979323846

        re_head, im_head = torch.chunk(head, 2, dim=2)
        re_tail, im_tail = torch.chunk(tail, 2, dim=2)

        # Make phases of relations uniformly distributed in [-pi, pi]

        phase_relation = relation / (self.embedding_range.item() / pi)

        re_relation = torch.cos(phase_relation)
        im_relation = torch.sin(phase_relation)

        if mode == 'head-batch':
            re_score = re_relation * re_tail + im_relation * im_tail
            im_score = re_relation * im_tail - im_relation * re_tail
            re_score = re_score - re_head
            im_score = im_score - im_head
        else:
            re_score = re_head * re_relation - im_head * im_relation
            im_score = re_head * im_relation + im_head * re_relation
            re_score = re_score - re_tail
            im_score = im_score - im_tail

        score = torch.stack([re_score, im_score], dim=0)
        score = score.norm(dim=0)

        score = self.gamma.item() - score.sum(dim=2)
        return score

    def pRotatE(self, head, relation, tail, mode):
        pi = 3.14159262358979323846

        # Make phases of entities and relations uniformly distributed in [-pi, pi]

        phase_head = head / (self.embedding_range.item() / pi)
        phase_relation = relation / (self.embedding_range.item() / pi)
        phase_tail = tail / (self.embedding_range.item() / pi)

        if mode == 'head-batch':
            score = phase_head + (phase_relation - phase_tail)
        else:
            score = (phase_head + phase_relation) - phase_tail

        score = torch.sin(score)
        score = torch.abs(score)

        score = self.gamma.item() - score.sum(dim=2) * self.modulus
        return score

    @staticmethod
    def train_step(model, optimizer, train_iterator, train_rule_triple, args):
        '''
        A single train step. Apply back-propation and return the loss
        '''

        model.train()

        optimizer.zero_grad()

        positive_sample, negative_sample, subsampling_weight, mode = next(train_iterator)

        rule_triple, rule_p = next(train_rule_triple)

        if args.cuda:
            positive_sample = positive_sample.cuda()
            negative_sample = negative_sample.cuda()
            subsampling_weight = subsampling_weight.cuda()
            rule_triple = rule_triple.cuda()
            rule_p = rule_p.cuda()

        negative_score = model((positive_sample, negative_sample), mode=mode)

        if args.negative_adversarial_sampling:
            # In self-adversarial sampling, we do not apply back-propagation on the sampling weight
            negative_score = (F.softmax(negative_score * args.adversarial_temperature, dim=1).detach()
                              * F.logsigmoid(-negative_score)).sum(dim=1)
        else:
            negative_score = F.logsigmoid(-negative_score).mean(dim=1)

        positive_score = model(positive_sample)

        rule_triple_score = model(rule_triple)

        positive_score = F.logsigmoid(positive_score).squeeze(dim=1)

        rule_triple_score = torch.sigmoid(rule_triple_score).squeeze(dim=1)

        if args.uni_weight:
            positive_sample_loss = - positive_score.mean()
            negative_sample_loss = - negative_score.mean()
        else:
            positive_sample_loss = - (subsampling_weight * positive_score).sum() / subsampling_weight.sum()
            negative_sample_loss = - (subsampling_weight * negative_score).sum() / subsampling_weight.sum()

        loss = (positive_sample_loss + negative_sample_loss) / 2

        if args.regularization != 0.0:
            # Use L3 regularization
            # regularization = args.regularization * (
            #         model.entity_embedding.norm(p=3) ** 3 +
            #         model.relation_embedding.norm(p=3).norm(p=3) ** 3
            # )

            # Use L2 regularization
            regularization = args.regularization * (
                    model.entity_embedding.norm(p=2) ** 2 / model.entity_embedding.size()[-1]
                    + model.relation_embedding.norm(p=2).norm(p=2) ** 2 / model.relation_embedding.size()[-1]
            )

            loss = loss + regularization
            regularization_log = {'regularization': regularization.item()}
        else:
            regularization_log = {}

        if args.use_rule:
            first = - ((rule_triple_score - 0.5) ** 2).mean()
            last = (rule_triple_score.mean() - rule_p.mean()) ** 2
            rule_confidence_loss = args.use_rule_coefficient * (0.25 - first + last)
            loss = loss + rule_confidence_loss
            rule_confidence_loss_log = {
                'rule triple score:': str(rule_triple_score),
                'rule triple score mean:': rule_triple_score.mean().item(),
                'confidence loss first item': first.item(),
                'confidence loss last item': last.item(),
                'rule_confidence_loss': rule_confidence_loss.item(),
            }
        else:
            rule_confidence_loss_log = {}

        loss.backward()
        optimizer.step()

        if args.NNE:
            for name, param in model.named_parameters():
                if name == 'entity_embedding':
                    param = torch.clamp(param, min=0.0, max=1.0)

        log = {
            **rule_confidence_loss_log,
            **regularization_log,
            'positive_sample_loss': positive_sample_loss.item(),
            'negative_sample_loss': negative_sample_loss.item(),
            'loss': loss.item()
        }

        return log


    @staticmethod
    def test_step(model, test_triples, all_true_triples, args, id2entity, id2relation):
        '''
        Evaluate the model on test or valid datasets
        '''

        model.eval()

        if args.countries:
            # Countries S* datasets are evaluated on AUC-PR
            # Process test data for AUC-PR evaluation
            sample = list()
            y_true = list()
            for head, relation, tail in test_triples:
                for candidate_region in args.regions:
                    y_true.append(1 if candidate_region == tail else 0)
                    sample.append((head, relation, candidate_region))

            sample = torch.LongTensor(sample)
            if args.cuda:
                sample = sample.cuda()

            with torch.no_grad():
                y_score = model(sample).squeeze(1).cpu().numpy()

            y_true = np.array(y_true)

            # average_precision_score is the same as auc_pr
            auc_pr = average_precision_score(y_true, y_score)

            metrics = {'auc_pr': auc_pr}

        else:
            # Otherwise use standard (filtered) MRR, MR, HITS@1, HITS@3, and HITS@10 metrics
            # Prepare dataloader for evaluation
            test_dataloader_head = DataLoader(
                TestDataset(
                    test_triples,
                    all_true_triples,
                    args.nentity,
                    args.nrelation,
                    'head-batch'
                ),
                batch_size=args.test_batch_size,
                num_workers=max(1, args.cpu_num // 2),
                collate_fn=TestDataset.collate_fn
            )

            test_dataloader_tail = DataLoader(
                TestDataset(
                    test_triples,
                    all_true_triples,
                    args.nentity,
                    args.nrelation,
                    'tail-batch'
                ),
                batch_size=args.test_batch_size,
                num_workers=max(1, args.cpu_num // 2),
                collate_fn=TestDataset.collate_fn
            )

            test_dataset_list = [test_dataloader_head, test_dataloader_tail]

            logs = []

            step = 0
            total_steps = sum([len(dataset) for dataset in test_dataset_list])

            head_triple = []
            tail_triple = []

            with torch.no_grad():
                for test_dataset in test_dataset_list:
                    for positive_sample, negative_sample, filter_bias, mode in test_dataset:
                        if args.cuda:
                            positive_sample = positive_sample.cuda()
                            negative_sample = negative_sample.cuda()
                            filter_bias = filter_bias.cuda()

                        batch_size = positive_sample.size(0)

                        score = model((positive_sample, negative_sample), mode)
                        score += filter_bias

#                         score = torch.clamp(score - args.margin, min=0.0)  # margin-based ranking loss

                        # Explicitly sort all the entities to ensure that there is no test exposure bias
                        argsort = torch.argsort(score, dim=1, descending=True)

                        if mode == 'head-batch':
                            positive_arg = positive_sample[:, 0]
                        elif mode == 'tail-batch':
                            positive_arg = positive_sample[:, 2]
                        else:
                            raise ValueError('mode %s not supported' % mode)

                        for i in range(batch_size):
                            # Notice that argsort is not ranking
                            ranking = (argsort[i, :] == positive_arg[i]).nonzero()
                            assert ranking.size(0) == 1

                            # ranking + 1 is the true ranking used in evaluation metrics
                            ranking = 1 + ranking.item()


                            if ranking > 1:
                                top10 = argsort[i, :][:10].tolist()
                                top10_e = []
                                for top in top10:
                                    top10_e.append(id2entity[top])
                                if mode == 'head-batch':
                                    right = [id2entity[positive_sample[i][0].item()],
                                             id2relation[positive_sample[i][1].item()],
                                             id2entity[positive_sample[i][2].item()], int(ranking)]
                                    tuple1 = right + top10_e
                                    head_triple.append(tuple1)
                                elif mode == 'tail-batch':
                                    right = [id2entity[positive_sample[i][0].item()],
                                             id2relation[positive_sample[i][1].item()],
                                             id2entity[positive_sample[i][2].item()], int(ranking)]
                                    tuple2 = right + top10_e
                                    tail_triple.append(tuple2)

                            logs.append({
                                'MRR': 1.0 / ranking,
                                'MR': float(ranking),
                                'HITS@1': 1.0 if ranking <= 1 else 0.0,
                                'HITS@3': 1.0 if ranking <= 3 else 0.0,
                                'HITS@10': 1.0 if ranking <= 10 else 0.0,
                            })

                        if step % args.test_log_steps == 0:
                            logging.critical('Evaluating the model... (%d/%d)' % (step, total_steps))

                        step += 1

            metrics = {}
            for metric in logs[0].keys():
                metrics[metric] = sum([log[metric] for log in logs]) / len(logs)

        return metrics, head_triple, tail_triple

Dataloader

In [ ]:
#!/usr/bin/python3

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch

from torch.utils.data import Dataset

class TrainDataset(Dataset):
    def __init__(self, triples, new_triple_only, nentity, nrelation, negative_sample_size, mode):
        self.len = len(triples)
        self.triples = triples
        self.triples_and_new_triple = triples + new_triple_only
        self.triple_set = set(triples)
        self.nentity = nentity
        self.nrelation = nrelation
        self.negative_sample_size = negative_sample_size
        self.mode = mode
        self.count = self.count_frequency(triples)


        self.true_head, self.true_tail = self.get_true_head_and_tail(self.triples_and_new_triple)
        # self.true_head, self.true_tail = self.get_true_head_and_tail(self.triples)




    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        positive_sample = self.triples[idx]

        head, relation, tail = positive_sample

        subsampling_weight = self.count[(head, relation)] + self.count[(tail, -relation-1)]
        subsampling_weight = torch.sqrt(1 / torch.Tensor([subsampling_weight]))

        negative_sample_list = []
        negative_sample_size = 0

        while negative_sample_size < self.negative_sample_size:
            negative_sample = np.random.randint(self.nentity, size=self.negative_sample_size*2)
            if self.mode == 'head-batch':
                mask = np.in1d(
                    negative_sample,
                    self.true_head[(relation, tail)],
                    assume_unique=True,
                    invert=True
                )
            elif self.mode == 'tail-batch':
                mask = np.in1d(
                    negative_sample,
                    self.true_tail[(head, relation)],
                    assume_unique=True,
                    invert=True
                )
            else:
                raise ValueError('Training batch mode %s not supported' % self.mode)
            negative_sample = negative_sample[mask]
            negative_sample_list.append(negative_sample)
            negative_sample_size += negative_sample.size

        negative_sample = np.concatenate(negative_sample_list)[:self.negative_sample_size]

        negative_sample = torch.LongTensor(negative_sample)

        positive_sample = torch.LongTensor(positive_sample)

        return positive_sample, negative_sample, subsampling_weight, self.mode

    @staticmethod
    def collate_fn(data):
        positive_sample = torch.stack([_[0] for _ in data], dim=0)
        negative_sample = torch.stack([_[1] for _ in data], dim=0)
        subsample_weight = torch.cat([_[2] for _ in data], dim=0)
        mode = data[0][3]
        return positive_sample, negative_sample, subsample_weight, mode

    @staticmethod
    def count_frequency(triples, start=4):
        '''
        Get frequency of a partial triple like (head, relation) or (relation, tail)
        The frequency will be used for subsampling like word2vec
        '''
        count = {}
        for head, relation, tail in triples:
            if (head, relation) not in count:
                count[(head, relation)] = start
            else:
                count[(head, relation)] += 1

            if (tail, -relation-1) not in count:
                count[(tail, -relation-1)] = start
            else:
                count[(tail, -relation-1)] += 1
        return count

    @staticmethod
    def get_true_head_and_tail(triples):
        '''
        Build a dictionary of true triples that will
        be used to filter these true triples for negative sampling
        '''

        true_head = {}
        true_tail = {}

        for head, relation, tail in triples:
            if (head, relation) not in true_tail:
                true_tail[(head, relation)] = []
            true_tail[(head, relation)].append(tail)
            if (relation, tail) not in true_head:
                true_head[(relation, tail)] = []
            true_head[(relation, tail)].append(head)

        for relation, tail in true_head:
            true_head[(relation, tail)] = np.array(list(set(true_head[(relation, tail)])))
        for head, relation in true_tail:
            true_tail[(head, relation)] = np.array(list(set(true_tail[(head, relation)])))

        return true_head, true_tail


class RuleTriple(Dataset):
    def __init__(self, triples, nentity, nrelation):
        self.len = len(triples)
        self.triples = triples
        self.nentity = nentity
        self.nrelation = nrelation

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        head, relation, tail, p = self.triples[idx]
        positive_sample = torch.LongTensor((head, relation, tail))
        p = torch.FloatTensor((p,))

        return positive_sample, p

    @staticmethod
    def collate_fn(data):
        positive_sample = torch.stack([_[0] for _ in data], dim=0)
        p = torch.stack([_[1] for _ in data], dim=0)
        return positive_sample, p





class TestDataset(Dataset):
    def __init__(self, triples, all_true_triples, nentity, nrelation, mode):
        self.len = len(triples)
        self.triple_set = set(all_true_triples)
        self.triples = triples
        self.nentity = nentity
        self.nrelation = nrelation
        self.mode = mode

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        head, relation, tail = self.triples[idx]

        if self.mode == 'head-batch':
            tmp = [(0, rand_head) if (rand_head, relation, tail) not in self.triple_set
                   else (-1, head) for rand_head in range(self.nentity)]
            tmp[head] = (0, head)
        elif self.mode == 'tail-batch':
            tmp = [(0, rand_tail) if (head, relation, rand_tail) not in self.triple_set
                   else (-1, tail) for rand_tail in range(self.nentity)]
            tmp[tail] = (0, tail)
        else:
            raise ValueError('negative batch mode %s not supported' % self.mode)

        tmp = torch.LongTensor(tmp)
        filter_bias = tmp[:, 0].float()
        negative_sample = tmp[:, 1]

        positive_sample = torch.LongTensor((head, relation, tail))

        return positive_sample, negative_sample, filter_bias, self.mode

    @staticmethod
    def collate_fn(data):
        positive_sample = torch.stack([_[0] for _ in data], dim=0)
        negative_sample = torch.stack([_[1] for _ in data], dim=0)
        filter_bias = torch.stack([_[2] for _ in data], dim=0)
        mode = data[0][3]
        return positive_sample, negative_sample, filter_bias, mode

class BidirectionalOneShotIterator(object):
    def __init__(self, dataloader_head, dataloader_tail):
        self.iterator_head = self.one_shot_iterator(dataloader_head)
        self.iterator_tail = self.one_shot_iterator(dataloader_tail)
        self.step = 0

    def __next__(self):
        self.step += 1
        if self.step % 2 == 0:
            data = next(self.iterator_head)
        else:
            data = next(self.iterator_tail)
        return data

    @staticmethod
    def one_shot_iterator(dataloader):
        '''
        Transform a PyTorch Dataloader into python iterator
        '''
        while True:
            for data in dataloader:
                yield data


class Iterator(object):
    def __init__(self, dataloader):
        self.iterator = self.iterator(dataloader)
        self.step = 0

    def __next__(self):
        self.step += 1
        data = next(self.iterator)
        return data

    @staticmethod
    def iterator(dataloader):
        '''
        Transform a PyTorch Dataloader into python iterator
        '''
        while True:
            for data in dataloader:
                yield data

Pipeline

In [ ]:
import argparse

if __name__ == "__main__":
    # arguments setting



    i = 0
    new_conclusions = []
    # m_n_s(args, tsv_source_path=args.row_triples, amie_path=args.amie_plus_path, minhc=args.minhc, minc=args.minc, minpca=args.minpca)
    # preprocess()
    # rule_reasoning(args)
    runKGE(args)

CRITICAL:root:Model: RotatE
2024-05-02 10:46:05,893 CRITICAL Model: RotatE
CRITICAL:root:Data Path: /content/drive/MyDrive/Bachelor/FB15k
2024-05-02 10:46:05,895 CRITICAL Data Path: /content/drive/MyDrive/Bachelor/FB15k
CRITICAL:root:#entity: 14951
2024-05-02 10:46:05,897 CRITICAL #entity: 14951
CRITICAL:root:#relation: 1345
2024-05-02 10:46:05,901 CRITICAL #relation: 1345


Model RotatE


CRITICAL:root:#train: 483142
2024-05-02 10:46:08,149 CRITICAL #train: 483142
CRITICAL:root:#valid: 50000
2024-05-02 10:46:09,000 CRITICAL #valid: 50000
CRITICAL:root:#test: 59071
2024-05-02 10:46:10,179 CRITICAL #test: 59071
CRITICAL:root:Model Parameter Configuration:
2024-05-02 10:46:27,626 CRITICAL Model Parameter Configuration:
CRITICAL:root:Parameter gamma: torch.Size([1]), require_grad = False
2024-05-02 10:46:27,629 CRITICAL Parameter gamma: torch.Size([1]), require_grad = False
CRITICAL:root:Parameter embedding_range: torch.Size([1]), require_grad = False
2024-05-02 10:46:27,632 CRITICAL Parameter embedding_range: torch.Size([1]), require_grad = False
CRITICAL:root:Parameter entity_embedding: torch.Size([14951, 2048]), require_grad = True
2024-05-02 10:46:27,634 CRITICAL Parameter entity_embedding: torch.Size([14951, 2048]), require_grad = True
CRITICAL:root:Parameter relation_embedding: torch.Size([1345, 1024]), require_grad = True
2024-05-02 10:46:27,638 CRITICAL Parameter re

My variable list:  {'step': 0, 'current_learning_rate': 0.0001, 'warm_up_steps': 7500}


CRITICAL:root:Training average rule triple score mean: at step 0: 0.054947
2024-05-02 10:47:42,197 CRITICAL Training average rule triple score mean: at step 0: 0.054947
CRITICAL:root:Training average confidence loss first item at step 0: -0.200956
2024-05-02 10:47:42,199 CRITICAL Training average confidence loss first item at step 0: -0.200956
CRITICAL:root:Training average confidence loss last item at step 0: 0.611343
2024-05-02 10:47:42,201 CRITICAL Training average confidence loss last item at step 0: 0.611343
CRITICAL:root:Training average rule_confidence_loss at step 0: 0.212460
2024-05-02 10:47:42,203 CRITICAL Training average rule_confidence_loss at step 0: 0.212460
CRITICAL:root:Training average regularization at step 0: 0.700432
2024-05-02 10:47:42,204 CRITICAL Training average regularization at step 0: 0.700432
CRITICAL:root:Training average positive_sample_loss at step 0: 3.150938
2024-05-02 10:47:42,206 CRITICAL Training average positive_sample_loss at step 0: 3.150938
CRIT

{'MRR': 0.006941663437831674, 'MR': 4784.17667, 'HITS@1': 0.00462, 'HITS@3': 0.00568, 'HITS@10': 0.0088}


CRITICAL:root:Training average rule triple score mean: at step 300: 0.500311
2024-05-02 12:15:16,300 CRITICAL Training average rule triple score mean: at step 300: 0.500311
CRITICAL:root:Training average confidence loss first item at step 300: -0.034245
2024-05-02 12:15:16,302 CRITICAL Training average confidence loss first item at step 300: -0.034245
CRITICAL:root:Training average confidence loss last item at step 300: 0.135566
2024-05-02 12:15:16,304 CRITICAL Training average confidence loss last item at step 300: 0.135566
CRITICAL:root:Training average rule_confidence_loss at step 300: 0.083962
2024-05-02 12:15:16,306 CRITICAL Training average rule_confidence_loss at step 300: 0.083962
CRITICAL:root:Training average regularization at step 300: 0.567137
2024-05-02 12:15:16,309 CRITICAL Training average regularization at step 300: 0.567137
CRITICAL:root:Training average positive_sample_loss at step 300: 0.812415
2024-05-02 12:15:16,312 CRITICAL Training average positive_sample_loss at

In [ ]:
# import torch

# checkpoint = torch.load('/content/drive/MyDrive/Bachelor/Results/checkpoint')

# # Iterate over the items in the checkpoint dictionary
# for key, value in checkpoint.items():
#     print(f"Key: {key}")
#     print(f"Type: {type(value)}")
#     print(f"Value: {value}")
#     print("\n")
